# Importanto Bibliotecas

In [2]:
import os
import cv2
import numpy as np
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

# Caminho das pastas
AUTHORIZED_DIR = 'authorized_faces'
EMBEDDING_DIR = 'embedding'

# Carrega o modelo `facenet`

In [3]:
# Detector de face e modelo de embeddings
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=160, margin=20, device=device)
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

100%|██████████| 107M/107M [00:48<00:00, 2.31MB/s] 


# Função para gerar embedding facial

In [6]:
def get_embedding(image_path):
    img = Image.open(image_path).convert('RGB')
    face = mtcnn(img)
    if face is None:
        raise ValueError(f"Nenhum rosto detectado em: {image_path}")
    face = face.unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = facenet(face)
    return embedding.squeeze().cpu().numpy()

# Função para calcular similaridade (cosine similarity)

In [7]:
def cosine_similarity(a, b):
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    return np.dot(a, b)

# Gera e salva os embeddings

In [8]:
# Gera e salva o embedding médio de cada usuário
def generate_mean_embeddings(authorized_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for user in os.listdir(authorized_dir):
        user_path = os.path.join(authorized_dir, user)
        if not os.path.isdir(user_path):
            continue
        embeddings = []
        for img_name in os.listdir(user_path):
            img_path = os.path.join(user_path, img_name)
            try:
                emb = get_embedding(img_path)
                embeddings.append(emb)
            except Exception as e:
                print(f"[Erro] {e}")
        if embeddings:
            mean_emb = np.mean(embeddings, axis=0)
            np.save(os.path.join(output_dir, f"{user}.npy"), mean_emb)
            print(f"✅ Embedding médio salvo: {user}")

# Load Savad Embeddings

In [9]:
def load_mean_embeddings(embedding_dir):
    embeddings = {}
    for file in os.listdir(embedding_dir):
        if file.endswith(".npy"):
            user = file.replace('.npy', '')
            vector = np.load(os.path.join(embedding_dir, file))
            embeddings[user] = vector
    return embeddings

# Verificar acesso comparando com múltiplas imagens por usuário

In [17]:
def verify_access(image_path, user_embeddings, threshold=0.7):
    try:
        input_embedding = get_embedding(image_path)
    except Exception as e:
        print(f"[Erro ao processar imagem de entrada] {e}")
        return False

    for user, emb_ref in user_embeddings.items():
        sim = cosine_similarity(input_embedding, emb_ref)
        # print(f"Comparando com {user} - Similaridade: {sim:.3f}")
        if sim > threshold:
            print(f"✅ Acesso Liberado para {user}")
            return True
    print("❌ Acesso Negado")
    return False

# Teste de verificação

In [ ]:
AUTHORIZED_DIR = 'authorized_faces'
EMBEDDING_DIR = 'mean_embeddings'

In [12]:
# Gerar os embeddings médios (execute quando atualizar imagens dos usuários)
generate_mean_embeddings(AUTHORIZED_DIR, EMBEDDING_DIR)

✅ Embedding médio salvo: Jonathan Breck
✅ Embedding médio salvo: Josh Radnor
✅ Embedding médio salvo: Konstantin Khabenskiy
✅ Embedding médio salvo: Kristen Wiig
✅ Embedding médio salvo: Sharon Leal
✅ Embedding médio salvo: Jenn Proske
✅ Embedding médio salvo: Bella Thorne
✅ Embedding médio salvo: Zach Galifianakis
✅ Embedding médio salvo: Franka Potente
✅ Embedding médio salvo: Sarah Jessica Parker
✅ Embedding médio salvo: Zack Ward
✅ Embedding médio salvo: Tiffany Haddish
✅ Embedding médio salvo: Jack Black
✅ Embedding médio salvo: Alice Englert
✅ Embedding médio salvo: Jonathan D Mellor
✅ Embedding médio salvo: Jeremy Sumpter
✅ Embedding médio salvo: Marion Cotillard
✅ Embedding médio salvo: Diane Lane
✅ Embedding médio salvo: Emily Mortimer
✅ Embedding médio salvo: Joan Allen
✅ Embedding médio salvo: Lubna Azabal
✅ Embedding médio salvo: Tahar Rahim
✅ Embedding médio salvo: Harrison Ford
✅ Embedding médio salvo: Jason Drucker
✅ Embedding médio salvo: Bel Powley
✅ Embedding médio sa

In [13]:
# Carregar embeddings salvos e verificar
authorized_embeddings = load_mean_embeddings(EMBEDDING_DIR)

In [14]:
authorized_embeddings.keys()

dict_keys(['Angelina Jolie', 'Logan Lerman', 'Linda Hamilton', 'Ioan Gruffudd', 'Jay Hernandez', 'Russell Brand', 'Anna Paquin', 'Julia Louis-Dreyfus', 'Joel Edgerton', 'Brett Dalton', 'Matt Passmore', 'Daniella Alonso', 'Megan Fox', 'Kristen Bell', 'Josephine Langford', 'Adelaide Clemens', 'Natasha Calis', 'Elsie Fisher', 'Rosa Salazar', 'Paul Giamatti', 'Josh Brolin', 'Jimmy Bennett', 'Kate Bosworth', 'Kate Beckinsale', 'Rupert Everett', 'Toshio Furukawa', 'Mel Gibson', 'Octavia Spencer', 'Regina Hall', 'Shane Harper', 'Rhys Ifans', 'Viola Davis', 'Andy Samberg', 'Jonathan Breck', 'Scarlett Johansson', 'Emma Stone', 'Amanda Bynes', 'Jason Momoa', 'Carlos Alazraqui', 'Brooklynn Prince', 'Stephen Campbell Moore', 'Josh Zuckerman', 'John Cameron Mitchell', 'Nathan Fillion', 'Woody Allen', 'Claire Foy', 'Elisha Cuthbert', 'Michael Fassbender', 'Sarah Butler', 'Julie Andrews', 'Douglas Smith', 'Nat Wolff', 'John Cusack', 'Ashley Tisdale', 'Katie Featherston', 'Noomi Rapace', 'David Arquet

In [15]:
authorized_embeddings['Angelina Jolie'].shape

(512,)

In [18]:
TEST_IMAGE = 'test_images/Adam Sandler.jpg'
# Executa verificação
status = verify_access(TEST_IMAGE, authorized_embeddings, threshold=0.75)

✅ Acesso Liberado para Adam Sandler


In [19]:
TEST_IMAGE = 'test_images/Octavia Spencer.jpg'
# Executa verificação
status = verify_access(TEST_IMAGE, authorized_embeddings, threshold=0.75)

✅ Acesso Liberado para Octavia Spencer


In [20]:
TEST_IMAGE = 'test_images/James Franco.jpg'
# Executa verificação
status = verify_access(TEST_IMAGE, authorized_embeddings, threshold=0.75)

✅ Acesso Liberado para James Franco


In [21]:
TEST_IMAGE = 'test_images/octavia-spencer-20724237-1-402.jpg'
# Executa verificação
status = verify_access(TEST_IMAGE, authorized_embeddings, threshold=0.75)

✅ Acesso Liberado para Octavia Spencer
